In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization
import json
from pprint import pprint
sns.set()

In [2]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [3]:
df = pd.read_csv('../dataset/GOOG.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-02,959.979980,962.539978,947.840027,953.270020,953.270020,1283400
1,2017-10-03,954.000000,958.000000,949.140015,957.789978,957.789978,888300
2,2017-10-04,957.000000,960.390015,950.690002,951.679993,951.679993,952400
3,2017-10-05,955.489990,970.909973,955.179993,969.960022,969.960022,1213800
4,2017-10-06,966.700012,979.460022,963.359985,978.890015,978.890015,1173900


In [4]:
close = df.Close.values.tolist()
window_size = 30
skip = 5
l = len(close) - 1

In [5]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(layer_size, 1),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [6]:
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        model,
        money,
        max_buy,
        max_sell,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = get_state(close, 0, self.window_size + 1)
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self):
        initial_money = self.initial_money
        state = get_state(close, 0, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                print(
                    'day %d: buy %d units at price %f, total balance %f'
                    % (t, buy_units, total_buy, initial_money)
                )
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (t, sell_units, total_sell, invest, initial_money)
                )
            state = next_state

        invest = ((initial_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (initial_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.show()

In [7]:
def best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    model = Model(window_size, size_network, 3)
    agent = Agent(
        population_size,
        sigma,
        learning_rate,
        model,
        10000,
        5,
        5,
        skip,
        window_size,
    )
    try:
        agent.fit(10, 1000)
        return agent.es.reward_function(agent.es.weights)
    except:
        return 0

In [8]:
def find_best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    global accbest
    param = {
        'window_size': int(np.around(window_size)),
        'skip': int(np.around(skip)),
        'population_size': int(np.around(population_size)),
        'sigma': max(min(sigma, 1), 0.0001),
        'learning_rate': max(min(learning_rate, 0.5), 0.000001),
        'size_network': int(np.around(size_network)),
    }
    print('\nSearch parameters %s' % (param))
    investment = best_agent(**param)
    print('stop after 100 iteration with investment %f' % (investment))
    if investment > accbest:
        costbest = investment
    return investment

In [9]:
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (2, 50),
        'skip': (1, 15),
        'population_size': (1, 50),
        'sigma': (0.01, 0.99),
        'learning_rate': (0.000001, 0.49),
        'size_network': (10, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 50, acq = 'ei', xi = 0.0)

|   iter    |  target   | learni... | popula... |   sigma   | size_n... |   skip    | window... |
-------------------------------------------------------------------------------------------------

Search parameters {'window_size': 41, 'skip': 12, 'population_size': 44, 'sigma': 0.46137729366237473, 'learning_rate': 0.31105292767202336, 'size_network': 444}
time taken to train: 2.682731866836548 seconds
stop after 100 iteration with investment 0.395400
|  1        |  0.3954   |  0.3111   |  43.76    |  0.4614   |  444.0    |  12.09    |  40.84    |

Search parameters {'window_size': 16, 'skip': 10, 'population_size': 35, 'sigma': 0.7324832941595844, 'learning_rate': 0.09052210105621832, 'size_network': 938}
time taken to train: 3.0886778831481934 seconds
stop after 100 iteration with investment 0.000000
|  2        |  0.0      |  0.09052  |  35.22    |  0.7325   |  937.9    |  10.4     |  16.48    |

Search parameters {'window_size': 11, 'skip': 7, 'population_size': 34, 'sigma': 0.3576

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


time taken to train: 2.1516976356506348 seconds
stop after 100 iteration with investment 0.000000
|  13       |  0.0      |  0.4333   |  14.84    |  0.5422   |  761.7    |  5.114    |  28.5     |

Search parameters {'window_size': 15, 'skip': 6, 'population_size': 27, 'sigma': 0.3553869673875342, 'learning_rate': 0.36935002189613014, 'size_network': 215}
time taken to train: 0.05280923843383789 seconds
stop after 100 iteration with investment 1.203401
|  14       |  1.203    |  0.3694   |  26.57    |  0.3554   |  214.8    |  5.666    |  14.59    |

Search parameters {'window_size': 39, 'skip': 12, 'population_size': 13, 'sigma': 0.4589762904618531, 'learning_rate': 0.3811277894022996, 'size_network': 217}
stop after 100 iteration with investment 0.000000
|  15       |  0.0      |  0.3811   |  12.67    |  0.459    |  216.6    |  12.5     |  38.94    |

Search parameters {'window_size': 40, 'skip': 7, 'population_size': 38, 'sigma': 0.20792649416556708, 'learning_rate': 0.390177301579724

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


stop after 100 iteration with investment 0.000000
|  31       |  0.0      |  0.2018   |  23.66    |  0.277    |  582.4    |  7.401    |  34.19    |

Search parameters {'window_size': 40, 'skip': 6, 'population_size': 24, 'sigma': 0.27153361713490215, 'learning_rate': 0.21764218046375214, 'size_network': 573}


...

stop after 100 iteration with investment 0.000000
|  75       |  0.0      |  0.3057   |  15.14    |  0.01151  |  426.3    |  3.876    |  42.89    |

Search parameters {'window_size': 41, 'skip': 11, 'population_size': 44, 'sigma': 0.5454309923128964, 'learning_rate': 0.4788120013716578, 'size_network': 443}


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


stop after 100 iteration with investment 0.000000
|  76       |  0.0      |  0.4788   |  44.45    |  0.5454   |  443.2    |  10.55    |  40.97    |

Search parameters {'window_size': 37, 'skip': 7, 'population_size': 23, 'sigma': 0.22404601973464977, 'learning_rate': 0.3347104059832889, 'size_network': 575}


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


stop after 100 iteration with investment 0.000000
|  77       |  0.0      |  0.3347   |  22.66    |  0.224    |  575.2    |  7.464    |  37.36    |

Search parameters {'window_size': 15, 'skip': 11, 'population_size': 23, 'sigma': 0.8391549126448979, 'learning_rate': 0.2693055416640527, 'size_network': 622}
time taken to train: 1.4061954021453857 seconds
stop after 100 iteration with investment 0.000000
|  78       |  0.0      |  0.2693   |  23.24    |  0.8392   |  622.1    |  11.48    |  15.08    |

Search parameters {'window_size': 36, 'skip': 6, 'population_size': 26, 'sigma': 0.8325731050223231, 'learning_rate': 0.4258779825047773, 'size_network': 575}
time taken to train: 3.153632640838623 seconds
stop after 100 iteration with investment 0.000000
|  79       |  0.0      |  0.4259   |  25.83    |  0.8326   |  574.8    |  6.303    |  35.7     |

Search parameters {'window_size': 27, 'skip': 9, 'population_size': 37, 'sigma': 0.2962491630655576, 'learning_rate': 0.2784756710759564, '

In [10]:
bayes_res = str(NN_BAYESIAN.max)
bayes_res = json.loads(bayes_res.replace("'", "\""))['params']
bayes_res['window_size'] = int(round(bayes_res['window_size']))
bayes_res['skip'] = int(round(bayes_res['skip']))
bayes_res['population_size'] = int(round(bayes_res['population_size']))
bayes_res['size_network'] = int(round(bayes_res['size_network']))
pprint(bayes_res)

{'learning_rate': 0.1113141226186473,
 'population_size': 25.772558952162992,
 'sigma': 0.758114864840465,
 'size_network': 576.2208404433251,
 'skip': 5.537525034757552,
 'window_size': 37.016032190153865}


In [11]:
# this is my parameter
best_agent(30, 1, 15, 0.1, 0.03, 500)

time taken to train: 9.468717575073242 seconds


-7.060601830000015

In [12]:
# this is from bayesian
best_agent(bayes_res['window_size']), bayes_res['skip'], bayes_res['population_size'], bayes_res['sigma'], bayes_res['learning_rate'], bayes_res['size_network'])

0

In [13]:
# this is my parameter
model = Model(30, 500, 3)
agent = Agent(15, 0.1, 0.03, model, 10000, 5, 5, 1, 30)
agent.fit(500, 100)
agent.buy()

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


ValueError: cannot convert float NaN to integer

In [ ]:
# this is from bayesian
model = Model(20, 176, 3)
agent = Agent(bayes_res['population_size'], bayes_res['sigma'], bayes_res['learning_rate'], model, 10000, 5, 5, bayes_res['skip'], bayes_res['window_size'])
agent.fit(500, 100)
agent.buy()